In [ ]:
from bluesky import RunEngine
from bluesky.plans import scan
from bluesky.plan_stubs import mv, open_run, close_run
from bluesky.callbacks import LiveTable
from bluesky.preprocessors import relative_set_decorator, \
    reset_positions_decorator
from ophyd.sim import motor1, motor2, det1

RE = RunEngine()

## Decorators

In [ ]:
@bluesky_decorator
def custom_plan(*args):
    # Customized plan
    pass

custom_plan(*args)

In [ ]:
bluesky_decorator(custom_plan)(*args)

def bluesky_decorator(*args):
    yield from decorator_before()
    
    yield from custom_plan(*args)
    
    yield from decorator_after()

## relative_set_decorator

In [ ]:
help(relative_set_decorator)

In [ ]:
@relative_set_decorator([motor1])
def relative_movement():
    yield from open_run()
    
    yield from mv(motor1, -2)
    print(motor1.readback.get())
    
    yield from mv(motor1, 3)
    print(motor1.readback.get())
    
    yield from mv(motor1, 1.5)
    print(motor1.readback.get())
    
    yield from close_run()

RE(mv(motor1, 5))
RE(relative_movement(), LiveTable(['motor1']))

In [ ]:
@relative_set_decorator()
def relative_scan():
    yield from scan([det1], motor1, -5, 2, 10)

RE(mv(motor1, 5))
RE(relative_scan(), LiveTable(['motor1']))

## reset_positions_decorator

In [ ]:
help(reset_positions_decorator)

In [ ]:
@reset_positions_decorator([motor1])
def reset_m1_position_scan():
    yield from scan([det1], motor1, -10, 10, 10)
    yield from mv(motor2, 5)

In [ ]:
RE(mv(motor1, 0, motor2, 0))
RE(reset_m1_position_scan())
print(f"Motor 1: {motor1.readback.get()}")
print(f"Motor 2: {motor2.readback.get()}")

In [ ]:
@reset_positions_decorator([motor1, motor2])
def reset_m1_and_m2_position_scan():
    yield from scan([det1], motor1, -10, 10, 10)
    yield from mv(motor2, 5)

In [ ]:
RE(mv(motor1, 0, motor2, 0))
RE(reset_m2_position_scan())
print(f"Motor 1: {motor1.readback.get()}")
print(f"Motor 2: {motor2.readback.get()}")

## run_decorator

In [ ]:
help(run_decorator)

In [ ]:
@run_decorator()
def plan_run():
    yield from mv(motor1, -10)
    yield from trigger_and_read([det1, motor1])
    
    yield from mv(motor1, 0)
    yield from trigger_and_read([det1, motor1])
    
    yield from mv(motor1, 10)
    yield from trigger_and_read([det1, motor1])
    
RE(
    plan_run(), LiveTable(['det1', 'motor1']))

In [ ]:
finalize_decorator: try...finally helper